<a href="https://colab.research.google.com/github/kaushikData/Document-Classification-Using-Wikipedia-Data/blob/master/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and Preprocessing

In [0]:
%pylab inline
import nltk
nltk.download('wordnet')
import pandas as pd
import pickle as pk
from scipy import sparse as sp
from gensim.models import LdaModel
import json
from nltk.corpus import stopwords

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaushik_shakkari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
with open('Wiki-Analysis/base_data_preprocessed.json') as obj:
    df = json.load(obj)
df = pd.DataFrame(df)
df.head()

,text,category,category_id
0,wowow wowow is collective in peckham london ot...,Arts,0
1,1376 in literature,Arts,0
10,c møller architect arkitektfirmaet f møller in...,Arts,0
100,anne beatts anne beatts born february 25 1947 ...,Arts,0
1000,Performative text\n\nIn the philosophy of lang...,Humanities,1


In [0]:
docs = df['text']

In [0]:
docs = [doc.split() for doc in docs]

In [0]:
docs = [[token for token in doc if not token.isdigit() and token not in stopwords.words('english')] for doc in docs]

# Generating Bigram abd Trigram

In [0]:
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
           
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            
            docs[idx].append(token)

/Users/kaushik_shakkari/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 244249
Number of unique words after removing rare and common words: 38355


In [0]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [0]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 38355
Number of documents: 14545


# Topic Classification

In [0]:
# Set training parameters.
num_topics = 28
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/Users/kaushik_shakkari/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 1h 12min 32s, sys: 10min, total: 1h 22min 33s
Wall time: 15min 39s


In [0]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/Users/kaushik_shakkari/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
21    -0.143486  0.083487       1        1  6.761262
2     -0.038524  0.010211       2        1  6.056194
25     0.105917 -0.102321       3        1  5.753470
1     -0.225623 -0.070137       4        1  5.752680
6      0.019172  0.039408       5        1  4.855395
11     0.099593 -0.001508       6        1  4.741526
16    -0.009122 -0.101548       7        1  4.536732
9      0.146610  0.098980       8        1  4.268809
13    -0.141897 -0.153665       9        1  4.181205
18     0.101665 -0.226372      10        1  3.973187
7     -0.203799 -0.023948      11        1  3.919032
8      0.017860  0.027363      12        1  3.831394
12     0.158694 -0.094456      13        1  3.631958
20    -0.023907 -0.164447      14        1  3.505718
14     0.057879  0.212800      15        1  3.306293
4      0.231418 -0.002659      16        1  3.278610
15    -0.203654  0.070072      17        1  3.076995
19    -0.087234  0.172927      18        1  3.007612
10    -0.180791 -0.016726      19        1  2.810669
23     0.096514  0.015292      20        1  2.772825
0      0.103272 -0.036026      21        1  2.676739
3     -0.127241  0.162555      22        1  2.523802
5     -0.126350 -0.113017      23        1  2.483902
26    -0.062367  0.062633      24        1  2.104484
27     0.049643 -0.093340      25        1  1.895612
17     0.128002  0.112724      26        1  1.541710
24     0.138917 -0.001065      27        1  1.379920
22     0.118837  0.132782      28        1  1.372261, topic_info=      Category          Freq               Term         Total  loglift  \
3179   Default  18116.000000       united_state  18116.000000  30.0000   
7560   Default  11774.000000               bell  11774.000000  29.0000   
524    Default   9689.000000           language   9689.000000  28.0000   
210    Default  10278.000000               book  10278.000000  27.0000   
327    Default   9824.000000             school   9824.000000  26.0000   
48     Default   7595.000000               film   7595.000000  25.0000   
296    Default   7914.000000           new_york   7914.000000  24.0000   
1210   Default   8565.000000              woman   8565.000000  23.0000   
1421   Default   4919.000000             church   4919.000000  22.0000   
1003   Default   6955.000000          published   6955.000000  21.0000   
705    Default   6053.000000               word   6053.000000  20.0000   
227    Default   6794.000000            company   6794.000000  19.0000   
2393   Default   7022.000000              court   7022.000000  18.0000   
785    Default   8221.000000              right   8221.000000  17.0000   
352    Default   5930.000000              water   5930.000000  16.0000   
893    Default   6270.000000               game   6270.000000  15.0000   
568    Default   6638.000000         philosophy   6638.000000  14.0000   
1345   Default   9384.000000                law   9384.000000  13.0000   
201    Default   8649.000000           american   8649.000000  12.0000   
11569  Default   4113.000000   neo_confucianism   4113.000000  11.0000   
913    Default   5794.000000                god   5794.000000  10.0000   
7391   Default   5589.000000             energy   5589.000000   9.0000   
1447   Default  10203.000000         government  10203.000000   8.0000   
2172   Default   4738.000000            student   4738.000000   7.0000   
830    Default   7781.000000            century   7781.000000   6.0000   
20     Default   6845.000000              child   6845.000000   5.0000   
664    Default   7766.000000              study   7766.000000   4.0000   
496    Default   7138.000000              human   7138.000000   3.0000   
94     Default   7891.000000              party   7891.000000   2.0000   
4183   Default   3867.000000        human_right   3867.000000   1.0000   
...        ...           ...                ...           ...      ...   